# Island Equilibrium Model

This is a code to implement the equilibrium model of the on-grid island connection problem with several colaborative prosumers.

## Libraries

In [7]:
from pyomo.environ import *
from pyomo.opt import SolverFactory, SolverStatus
import numpy as np
import pandas as pd

## About the data

This model can work either with daily or hourly data. If the data is taken daily then the studied time horizon is a month, and the number of days studied depends on the month in question. If the data is taken hourly then the studied time horizon is a day (24h), and in this case a particular month has to be chosen and solutions for all the days will be provided. After the model is loaded, come back to the next cell (Master cell) and choose the period for which you want the solutions ot be obtained.

**Possible months range:**\
['April', 'August', 'December', 'February', 'January', 'July', 'June', 'March', 'May', 'November', 'October', 'September'] 

**Possible days ranges:**\
January, March, May, July, August, October, December:\
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31]

April, June, September, November:\
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30]

February:\
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28]

We also have data for 2019 and 2018, so you also need to indnicate the year you want to solve the model for.


# Master cell 
### for getting several solutions (run after model and rest of code)

In [ ]:
#Choose the timing:
timing = 'hourly'
#timing = 'daily'

#Choosing year
#year = '2019'
year = '2018'

#Choose month range (a list with all the months we are interested in):
#BC: I believe there's a predefined string with this (I use it in the subplots code, just do't remember it now)
months_range= ['April', 'August', 'December', 'February', 'January', 'July', 'June', 'March', 'May', 'November', 'October', 'September']   

several_solutions(timing, months_range, year)

### Function that iterates the process over months and days 

In [8]:
def several_solutions(timing, month_range, year):
    """
    This function solves and stores the solution
    for the months chosen in months_range. If
    the timing is hourly, one solution for each 
    day of the month is found and stored.
    """
    # Might be good to have a function to create the model as well.
    # Since it is an abstract model, we can create it outside the for loop below
    
    for m in month_range:
        if timing=='hourly':
            #BC: I'm almost sure there is a function implemented in python that does this (maybe in the library 'calendar')
            if m in ['January', 'March', 'May', 'July', 'August', 'October', 'December']:
                day_range = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31]
                #BC: Might be more readable to write: 'day_range = range(1,32)'
            elif m in ['April', 'June', 'September', 'November']:
                day_range = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30]
                #BC: day_range = range(1,31)
            else:
                day_range = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28]
                #BC: day_range = range(1,29)

            for d in day_range:
                instance= create_instance_from_data(timing,m,d, year)
                solve_model(instance)
                store_solution(timing, m, d, instance, year)
                export_system_solution(timing, m, d, instance, year)
                export_prosumer_solution(timing, m, d, instance, year)
                export_battery_solution(timing, m, d, instance, year)
            print('Solutions for ' + m + ' stored')
        
        else:
            d=None
            # BC: The next lines are repetead code from above. This usually means you can write a function for it
            instance= create_instance_from_data(timing, m, d, year)
            solve_model(instance)
            store_solution(timing, m, d, instance, year)
            export_system_solution(timing, m, d, instance, year)
            export_prosumer_solution(timing, m, d, instance, year)
            export_battery_solution(timing, m, d, instance, year)
            print('Solutions for ' + m + ' stored')
            

## Create Model

In [9]:
#Create the model
model = AbstractModel()

#Define Sets of Prosumers(I) and Time Periods(T)
model.num_prosumers    = Param(within = NonNegativeIntegers , mutable = True)
model.num_time_periods = Param(within = NonNegativeIntegers , mutable = True)
model.I = RangeSet(model.num_prosumers)
model.T = RangeSet(model.num_time_periods)

# Define Parameters 
model.market_price         = Param(model.T, within = NonNegativeReals, mutable = True)
model.pv_price             = Param(model.I, within = NonNegativeReals, mutable = True)
model.battery_capacity     = Param(model.I, within = NonNegativeReals, mutable = True)
model.pv_generation        = Param(model.I, model.T, within = NonNegativeReals, mutable = True)
model.demand               = Param(model.I, model.T, within = NonNegativeReals, mutable = True)
model.self_discharge_rate  = Param(model.I, within = NonNegativeReals, mutable = True)
model.efficiency_charge    = Param(model.I, within = NonNegativeReals, mutable = True)
model.efficiency_discharge = Param(model.I, within = NonNegativeReals, mutable = True)
model.inicial_soc_retailer = Param(model.I, within = NonNegativeReals, mutable = True)
model.inicial_soc_prosumer = Param(model.I, within = NonNegativeReals, mutable = True)
model.retailer_max_battery = Param(model.I, within = NonNegativeReals, mutable = True)
model.prosumer_max_battery = Param(model.I, within = NonNegativeReals, mutable = True)
model.peer_fee = Param(model.I, within = NonNegativeReals, mutable = True)
model.send_price = Param(model.I, within = NonNegativeReals, mutable = True)

#Define Variables
model.prosumer_bought      = Var(model.I, model.T, within = NonNegativeReals)
model.prosumer_sold        = Var(model.I, model.T, within = NonNegativeReals)
model.prosumer_sent        = Var(model.I, model.I, model.T, within = NonNegativeReals)
model.prosumer_received    = Var(model.I, model.I, model.T, within = NonNegativeReals)
model.prosumer_charged     = Var(model.I, model.T, within = NonNegativeReals)
model.prosumer_discharged  = Var(model.I, model.T, within = NonNegativeReals)
model.retailer_charged     = Var(model.I, model.T, within = NonNegativeReals)
model.retailer_discharged  = Var(model.I, model.T, within = NonNegativeReals)
model.retailer_soc         = Var(model.I, model.T, within = NonNegativeReals) 
model.prosumer_soc         = Var(model.I, model.T, within = NonNegativeReals)

In [10]:
#Define Objective Function and Constraints

#Objective function: Cost of operating (meeting demand) the whole system.
def operational_cost_island(model):
    """
    This fuction calculates the total cost of the system. 
    """
    cost_energy_bought = sum(model.market_price[t] * \
                             (model.prosumer_bought[i,t] + model.retailer_charged[i,t]) \
                             for t in model.T for i in model.I)
    profit_energy_sold = sum(model.market_price[t] * model.retailer_discharged[i,t] +\
                             model.pv_price[i] * model.prosumer_sold[i,t]
                             for t in model.T for i in model.I)
    cost_commission_p2p = sum(model.peer_fee[i] * \
                               (sum(model.send_price[i] * model.prosumer_sent[i,j,t] \
                                    + model.send_price[j] * model.prosumer_received[i,j,t] \
                                    for j in model.I)) \
                               for t in model.T for i in model.I)
    
    return(cost_energy_bought - profit_energy_sold + cost_commission_p2p)

#Constraints

# Initial, update and Final State-of-Charge of Prosumer's Share of the Battery
def soc_prosumer_update(model,i,t):
    """
    These constraints update the amount of electricity 
    at time t in the battery used by prosumer i.
    """
    if t == min(model.T):
        return(model.prosumer_soc[i,t] == model.inicial_soc_prosumer[i])
    else:
        return(model.prosumer_soc[i,t] == (1 - model.self_discharge_rate[i]) * model.prosumer_soc[i,t-1] \
                                        - model.prosumer_discharged[i,t-1] / model.efficiency_discharge[i] \
                                        + model.efficiency_charge[i] * model.prosumer_charged[i, t-1])
    
def final_soc_prosumer(model,i):
    """
    This constraint defines the final load of the battery
    to be the same as the initial state of charge.
    """
    t = max(model.T)
    final_soc_prosumer = (1 - model.self_discharge_rate[i]) * model.prosumer_soc[i,t]\
                          - model.prosumer_discharged[i,t] / model.efficiency_discharge[i]\
                          + model.efficiency_charge[i] * model.prosumer_charged[i,t]
    
    return(final_soc_prosumer == model.inicial_soc_prosumer[i])

# Initial, update and Final State-of-Charge of Retailer's Share of the Battery
def soc_retailer_update(model,i,t):
    """
    These constraints update the amount of electricity 
    at time t in the battery used by retailer of 
    prosumer i.
    """
    if t == min(model.T): 
        return(model.retailer_soc[i,t] == model.inicial_soc_retailer[i])
    else:
        return(model.retailer_soc[i,t] == (1 - model.self_discharge_rate[i]) * model.retailer_soc[i,t-1] \
                                       - model.retailer_discharged[i,t-1]/model.efficiency_discharge[i] \
                                       + model.efficiency_charge[i] * model.retailer_charged[i, t-1])
    
def final_soc_retailer(model,i):
    """
    This constraint defines the final load of the battery to be the same as the initial state of charge.
    """
    t = max(model.T)
    final_soc_prosumer = (1 - model.self_discharge_rate[i]) * model.retailer_soc[i,t] \
                          - model.retailer_discharged[i,t]/model.efficiency_discharge[i] \
                          + model.efficiency_charge[i] * model.retailer_charged[i,t]
    
    return(final_soc_prosumer == model.inicial_soc_retailer[i])

# Limits of the State-of-Charge of Retailer's and Prosumer's Share of the Battery
def retailer_soc_limit(model,i,t):
    """
    These constraints hold that the retailer cannot 
    store in prosumer i battery more than the allowed
    capacity.
    """
    return (model.retailer_soc[i,t] <= model.retailer_max_battery[i])

def prosumer_soc_limit(model,i,t):
    """
    These constraints hold that the prosumer cannot 
    store in battery more than the allowed
    capacity.
    """
    return (model.prosumer_soc[i,t] <= model.prosumer_max_battery[i])

# Limits on the Energy Prosumers Sell and Buy

def prosumer_sell_limit(model,i,t):
    """
    These constraints hold that prosumer i cannot sell more 
    electricity than the amount pv-generated. 
    """
    return(model.prosumer_sold[i,t] <= model.pv_generation[i,t])

def prosumer_buy_limit(model,i,t):
    """
    These constraints hold that prosumer i cannot buy more 
    electricity than its own demand.
    """
    return(model.prosumer_bought[i,t] <= model.demand[i,t])

# Demand/Supply Balance Equation
def demand_supply_balance(model,i,t):
    """
    This constraint assure that at all time periods t the
    demand of prosumer i is equal to the supply.
    """
    supply = model.pv_generation[i,t] + model.prosumer_bought[i,t] \
            + sum(model.prosumer_received[i,j,t] for j in model.I) \
            + model.prosumer_discharged[i,t]#/model.efficiency_discharge[i]
    demand = model.demand[i,t] + model.prosumer_sold[i,t] \
            + sum(model.prosumer_sent[i,j,t] for j in model.I) \
            +  model.prosumer_charged[i,t]#model.efficiency_charge[i] *
    
    return(supply == demand)

#Symmetry of Energy Sent between Prosumers
def symmetry_send_receive(model,i,j,t):
    """
    This constraint sets the energy sent from prosumer i to prosumer j to be
    equal to the energy received by prosumer j from prosumer i.
    """
    return(model.prosumer_received[i,j,t] == model.prosumer_sent[j,i,t])

def prosumer_send_itself(model,i,j,t):
    """
    This constraint sets the energy sent prosumers send to themselves to be
    equal to zero.
    """
    if i == j:
        return(model.prosumer_sent[i,j,t] == 0)
    else: 
        return(Constraint.Skip)

#Limits of Energy Sent between Prosumers
def limit_receive(model,i,t):
    """
    This constraint sets the limit on the energy sent from prosumer i to
    other prosumers at time t.
    """
    limit = model.demand[i,t] + model.prosumer_charged[i,t]
    return(sum(model.prosumer_received[i,j,t] for j in model.I) <= limit)

def limit_send(model,i,t):
    """
    This constraint sets the limit on the energy received by prosumer i from
    other prosumers at time t.
    """
    limit = model.pv_generation[i,t] + model.prosumer_discharged[i,t]
    return(sum(model.prosumer_sent[i,j,t] for j in model.I) <= limit)


# Declare Model's Objective and Constraints
#Objective Function-------------------------------------------------------------------------------------
model.obj_operational_cost = Objective(rule = operational_cost_island, sense = minimize)

#Constraints--------------------------------------------------------------------------------------------
model.cons_soc_prosumer_update = Constraint(model.I, model.T, rule = soc_prosumer_update)
model.cons_final_soc_prosumer = Constraint(model.I, rule = final_soc_prosumer)

model.cons_soc_retailer_update = Constraint(model.I, model.T, rule = soc_retailer_update)
model.cons_final_soc_retailer = Constraint(model.I, rule = final_soc_retailer)

model.cons_retailer_soc_limit = Constraint(model.I, model.T, rule = retailer_soc_limit)
model.cons_prosumer_soc_limit = Constraint(model.I, model.T, rule = prosumer_soc_limit)

model.cons_prosumer_sell_limit = Constraint(model.I, model.T, rule = prosumer_sell_limit)
model.cons_prosumer_buy_limit = Constraint(model.I, model.T, rule = prosumer_buy_limit)

model.cons_demand_supply_balance = Constraint(model.I, model.T, rule = demand_supply_balance)

model.cons_symmetry_send_receive = Constraint(model.I, model.I, model.T, rule = symmetry_send_receive)
model.cons_prosumer_send_itself = Constraint(model.I, model.I, model.T, rule = prosumer_send_itself)

model.cons_receive_limit = Constraint(model.I, model.T, rule = limit_receive)
model.cons_send_limit = Constraint(model.I, model.T, rule = limit_send)

## Read Data Instance

In [11]:
# Load Data from CSVs
def create_instance_from_data(timing,month,day, year):
    """ 
    This function createds an instance
    from desired data. This depends on
    whether data is hourly or daily and 
    on the date 
    """
    #BC: Why don't we do a small if else called folder name like this:
    # folder_name = 'Data/' + year + '/' + timing + '/'
    #
    # if timing == 'daily':
    #     folder_name = folder_name + month + '/'
    #BC: and then we don't need a big if else below and 
    #    we can just write filename = folder_name + whatever the excel file name is
    
    data = DataPortal()
    if timing == 'daily':
        data.load(filename = 'Data/' + year + '/daily/' + month + '/instance_size.csv', param = (model.num_prosumers, model.num_time_periods))
        data.load(filename = 'Data/' + year + '/daily/pv_price.csv', param = model.pv_price)
        data.load(filename = 'Data/' + year + '/daily/peer_data.csv', param = (model.send_price, model.peer_fee))
        data.load(filename = 'Data/' + year + '/daily/efficiencies_initial_max_soc.csv', param = (model.self_discharge_rate, model.efficiency_charge,\
                                                                           model.efficiency_discharge, model.inicial_soc_retailer,\
                                                                           model.inicial_soc_prosumer, model.retailer_max_battery,\
                                                                           model.prosumer_max_battery, model.battery_capacity))
        data.load(filename = 'Data/' + year + '/daily/' + month + '/market_price.csv', param = model.market_price)
        data.load(filename = 'Data/Pecan/daily/daily_generation/' + month + '_generation.csv', param = model.pv_generation, format='array')
        data.load(filename = 'Data/Pecan/daily/daily_demand/' + month + '_demand.csv', param = model.demand, format='array')

    else: #(timing == 'hourly')
        data.load(filename = 'Data/' + year + '/hourly/instance_size.csv', param = (model.num_prosumers, model.num_time_periods))
        data.load(filename = 'Data/' + year + '/hourly/pv_price.csv', param = model.pv_price)
        data.load(filename = 'Data/' + year + '/hourly/peer_data.csv', param = (model.send_price, model.peer_fee))
        data.load(filename = 'Data/' + year + '/hourly/efficiencies_initial_max_soc.csv', param = (model.self_discharge_rate, model.efficiency_charge,\
                                                                           model.efficiency_discharge, model.inicial_soc_retailer,\
                                                                           model.inicial_soc_prosumer, model.retailer_max_battery,\
                                                                           model.prosumer_max_battery, model.battery_capacity))
        data.load(filename = 'Data/' + year + '/hourly/' + month + '/' + str(day) + '_market_price.csv', param = model.market_price)
        data.load(filename = 'Data/Pecan/hourly/hourly_generation/' + month + '/' + str(day) + '_' + month + '_generation.csv', param = model.pv_generation, format='array')
        data.load(filename = 'Data/Pecan/hourly/hourly_demand/' + month + '/' + str(day) + '_' + month + '_demand.csv', param = model.demand, format='array')

    #Create Data Instance
    instance = model.create_instance(data)
    return(instance)

# To view data, type for example:
#data['pv_price']
#instance.pv_generation.display()

## Solve Model

In [12]:
# Define Solver and Solve Model Instance
def solve_model(instance):
    """
    This function solves the model for a 
    particular instance using gurobi solver.
    """
    opt = SolverFactory("gurobi", solver_io = "python")
    results = opt.solve(instance, load_solutions = False)

    # Optimality Check to save solutions
    if results.solver.termination_condition == TerminationCondition.optimal:
        instance.solutions.load_from(results)
    #BC: Let's add an else to print an error mesage and the termination message in case it is not optimal

## Store Solution

In [13]:
def store_solution(timing,month,day,instance, year):
    """
    This function stores the unprocessed general
    solution to the model in an excel file named
    'Solution'. 
    """
    # Define Solution's File Name
    if timing== 'daily':
        file_name = '/Solution.xlsx'
    else:
        file_name = '/' + str(day) + '_Solution.xlsx'
        
    # Open the Pandas Excel writer
    writer = pd.ExcelWriter('solution_files/' + year + '/' + timing + '_solutions/' + month + file_name, engine='xlsxwriter')

    # Iterate over variables in the model
    for var in instance.component_objects(Var, active=True): 
        # Store solutions to a dataframe 
        df = pd.DataFrame.from_dict(var.extract_values(), orient='index', columns=[str(var)])
        # Upload solutions to excel sheet with correspoding name
        df.to_excel(writer, sheet_name=str(var), index=True)

    # Close the Pandas Excel writer and output the Excel file
    writer.save()

## Exporting system solutions

In [14]:
# BC: Maybe we can use a shorter way to get all this values nicely in a dataframe like in `store_solution`,
# and avoid having to iterate over all indexes
# Otrherwise: shall we have 1 single loop over T where we do everthing?

def export_system_solution(timing,month,day,instance, year):
    """
    This is a function that stored in an excel
    file the syatem's solution, summing over prosumers
    and retailers' solutions.
    """
    #List of all the names of the values we want to save.    
    column_names=['System demand', 'System PV generation', 'System sold to grid', 'System bought from grid']

    #Creating a data frame with our rows and column names.
    df_system= pd.DataFrame(columns= column_names, index= instance.T)

    # Adding total demand to dataframe.
    #Creating a list with the total demand of the system (sum of all prosumers).
    list_system_demand=[] #Creates an empty list to latter fill with the demand of all prosumers at each time period.
    for t in instance.T:
        list_system_demand.append(sum(instance.demand[(k,t)].value for k in instance.I)) #Adds the total demand to the list.
    df_system['System demand']= list_system_demand #Makes the column of our data frame equal to list.

    #Adding total generation to dataframe.
    #Creating a list with the total generation of the system (sum of all prosumers).
    list_electricity_pv_generation=[] 
    for t in instance.T:
        list_electricity_pv_generation.append(sum(instance.pv_generation[(k,t)].value for k in instance.I)) #Adds the total generation to the list.
    df_system['System PV generation']= list_electricity_pv_generation 

    #Adding price of total electricity sold.
    #Creating a list with the total electricity sold by the system (sum of all prosumers selling their excess electricity at the feed-in price, 
    #plus the amount sold by the retailer from batteries at the market price).
    list_prosumer_electricity_sold=[] 
    for t in instance.T:
        list_prosumer_electricity_sold.append(sum(instance.pv_price[k].value * instance.prosumer_sold[(k,t)].value for k in instance.I) +  #Adding the sum over all prosumers at each time period of sold electricity to grid.
                                              sum(instance.market_price[t].value * instance.retailer_discharged[(k,t)].value for k in instance.I)) #Adding the sum over all prosumers' batteries of the discharged electricity by retailer sold at market price.
    df_system['System sold to grid']= list_prosumer_electricity_sold 

    #Adding price of total electricity bought.
    #Creating a list with the total electricity bought by the system (sum of all prosumers' bought electricity at the market price, plus the 
    #electricity bought by the retailer to charge the batteries at market price).
    list_electricity_bought=[]
    for t in instance.T:
        list_electricity_bought.append(sum(instance.market_price[t].value * instance.prosumer_bought[(k,t)].value for k in instance.I) +  #Adding the sum over all prosumers at each time period of bought electricity.
                                       sum(instance.market_price[t].value * instance.retailer_charged[(k,t)].value for k in instance.I)) #Adding the sum over all prosumers' batteries of the charged (bought) electricity by retailer at market price.
    df_system['System bought from grid']= list_electricity_bought

    # Store data to an excel file.
    #Define file name.
    if timing== 'daily':
        file_name = '/System_solutions.xlsx'
    else:
        file_name = '/' + str(day) + '_System_solutions.xlsx'
    
    writer = pd.ExcelWriter('solution_files/' + year + '/' + timing + '_solutions/' + month + file_name, engine='xlsxwriter')
    df_system.to_excel(writer, sheet_name='System', index=False)
    writer.save() # Close the Pandas Excel writer and output the Excel file.

## Exporting prosumers solution

In [15]:
def export_prosumer_solution(timing,month,day,instance, year):
    """
    This function stores each prosumer's solution
    in a sheet called 'Prosumeri' in an excel file
    names 'Prosumers_solutions' for each month and 
    day.
    """
    #List of all the names of the values we want to save.        
    column_names=['Electricity bought from grid', 'PV generation', 'Electricity discharged from battery',
                'Electricity received from other prosumers', 'Electricity sent to other prosumers', 
                'Electricity charged to battery', 'Electricity sold to the grid', 'Electricity demand']

    #Creating a list of data frames (one for each prosumer) with the output data.
    list_of_data_frames=[] 
    for k in instance.I:
        #Creating the data frame with desired rows and columns
        df_prosumer= pd.DataFrame(columns= column_names, index= instance.T)

        #Creating a list of the electricity bought by the prosumer at each time period to assigned it to its df column.
        list_prosumer_electricity_bought=[] #Creates an empty list to latter fill with the electricity bought by prosumer k.
        for t in instance.T:
            list_prosumer_electricity_bought.append(instance.prosumer_bought[(k,t)].value) #Adds all the bought electricity values to the list.
        df_prosumer['Electricity bought from grid']= list_prosumer_electricity_bought #Makes the column of our data frame equal to list.

        #Adding generation to dataframe.
        list_electricity_pv_generation=[]
        for t in instance.T:
            list_electricity_pv_generation.append(instance.pv_generation[(k,t)].value) #Adds the generation over time period to list.
        df_prosumer['PV generation']= list_electricity_pv_generation

        #Adding amount discharged to dataframe.
        list_prosumer_electricity_discharged_battery=[]
        for t in instance.T:
            list_prosumer_electricity_discharged_battery.append(instance.prosumer_discharged[(k,t)].value) #Adds the electricity discharged at each period.
        df_prosumer['Electricity discharged from battery']= list_prosumer_electricity_discharged_battery

        #Adding total received electricity.
        list_prosumer_received_electricity=[]
        for t in instance.T:
            list_prosumer_received_electricity.append(sum(instance.prosumer_received[(k,j,t)].value for j in instance.I)) #We are summing the electricity received by all other prosumers.
        df_prosumer['Electricity received from other prosumers']= list_prosumer_received_electricity

        #Adding sent electricity.
        list_prosumer_sent_electricity=[]
        for t in instance.T:
            list_prosumer_sent_electricity.append(sum(instance.prosumer_sent[(k,j,t)].value for j in instance.I)) #We are summing the electricity sent to all other prosumers.
        df_prosumer['Electricity sent to other prosumers']= list_prosumer_sent_electricity

        #Adding electricity charged to battery.
        list_prosumer_electricity_charged_battery=[]
        for t in instance.T:
            list_prosumer_electricity_charged_battery.append(instance.prosumer_charged[(k,t)].value) #Adds the electricity charged at each time period.
        df_prosumer['Electricity charged to battery']= list_prosumer_electricity_charged_battery

        #Adding sold electricity.
        list_prosumer_electricity_sold=[]
        for t in instance.T:
            list_prosumer_electricity_sold.append(instance.prosumer_sold[(k,t)].value) #Adds the electricity sold at each time period.
        df_prosumer['Electricity sold to the grid']= list_prosumer_electricity_sold

        #Adding electricity demand.
        list_electricity_demand=[]
        for t in instance.T:
            list_electricity_demand.append(instance.demand[(k,t)].value) #Adds the demand at each time period.
        df_prosumer['Electricity demand']= list_electricity_demand

        list_of_data_frames.append(df_prosumer) #Adds each prosumer's data frame to our list of data frames.
        
    # Store each prosumers data to an excel sheet.
    #Define file name.
    if timing== 'daily':
        file_name = '/Prosumers_solutions.xlsx'
    else:
        file_name = '/' + str(day) + '_Prosumers_solutions.xlsx'
   
    writer = pd.ExcelWriter('solution_files/' + year + '/' + timing + '_solutions/' + month + file_name, engine='xlsxwriter')
    #we now iterate over each dataframe of the lost of data frames and add them to the excel file as a sheet.
    for i in instance.I:
        list_of_data_frames[i-1].to_excel(writer, sheet_name='Prosumer'+str(i), index=False)
    writer.save() # Close the Pandas Excel writer and output the Excel file.

## Exporting batteries information

In [16]:
def export_battery_solution(timing, month, day, instance, year):
    """
    This function stores the state of charge of each
    battery and the batteries operations in an excel 
    file called 'Batteries_prosumers_solutions' and 
    a sheet for each prosumer'
    """
    #List of all the names of the values we want to save.    
    column_names2=['Prosumer battery load', 'Prosumer battery charged', 'Prosumer battery discharged','Retailer battery load', 
                  'Retailer battery charged', 'Retailer battery discharged', 'Total battery load', 'Total battery charged',
                  'Total battery discharged', 'Prosumer max battery allowance', 'Retailer max battery allowance']

    #Creating a list of data frames (one for each prosumer) with the output data.
    list_of_data_frames=[]
    for p in instance.I:
        df_prosumer= pd.DataFrame(columns= column_names2, index= instance.T)

        # Adding prosumer battery load to the data frame.
        #Creating a list with the load at each time period.
        list_prosumer_battery_load=[] #Creates an empty list to latter fill with the load at each time period.
        for t in instance.T:
            list_prosumer_battery_load.append(instance.prosumer_soc[(p,t)].value) #Adds the prosumer load to the list.
        df_prosumer['Prosumer battery load']= list_prosumer_battery_load #Makes the column of our data frame equal to list.

        #Adding prosumer electricity charged to dataframe.
        list_prosumer_battery_charged=[]
        for t in instance.T:
            list_prosumer_battery_charged.append(instance.prosumer_charged[(p,t)].value) #Adds the charged electricity to the list.
        df_prosumer['Prosumer battery charged']= list_prosumer_battery_charged 

        #Adding prosumer electricity discharged.
        list_prosumer_battery_discharged=[]
        for t in instance.T:
            list_prosumer_battery_discharged.append(instance.prosumer_discharged[(p,t)].value) #Adding the prosumer discharged electricity.
        df_prosumer['Prosumer battery discharged']= list_prosumer_battery_discharged

        # Adding retailer battery load to the data frame.
        list_retailer_battery_load=[] 
        for t in instance.T:
            list_retailer_battery_load.append(instance.retailer_soc[(p,t)].value) #Adds the retailer load to the list.
        df_prosumer['Retailer battery load']= list_retailer_battery_load

        #Adding retailer electricity charged to dataframe.
        list_retailer_battery_charged=[]
        for t in instance.T:
            list_retailer_battery_charged.append(instance.retailer_charged[(p,t)].value) #Adds the charged electricity to the list.
        df_prosumer['Retailer battery charged']= list_retailer_battery_charged 

        #Adding retailer electricity discharged to dataframe.
        list_retailer_battery_discharged=[]
        for t in instance.T:
            list_retailer_battery_discharged.append(instance.retailer_discharged[(p,t)].value) #Adds the discharged electricity to the list.
        df_prosumer['Retailer battery discharged']= list_retailer_battery_discharged 

         # Adding total battery load to the data frame.
        list_total_battery_load=[] 
        for t in instance.T:
            list_total_battery_load.append(instance.retailer_soc[(p,t)].value + instance.prosumer_soc[(p,t)].value) #Adds the total load to the list.
        df_prosumer['Total battery load']= list_total_battery_load 

        #Adding total electricity charged to dataframe.
        list_total_battery_charged=[]
        for t in instance.T:
            list_total_battery_charged.append(instance.retailer_charged[(p,t)].value + instance.prosumer_charged[(p,t)].value) #Adds the total charged electricity to the list.
        df_prosumer['Total battery charged']= list_total_battery_charged 

        #Adding total electricity discharged to dataframe.
        list_total_battery_discharged=[]
        for t in instance.T:
            list_total_battery_discharged.append(instance.retailer_discharged[(p,t)].value + instance.prosumer_discharged[(p,t)].value) #Adds the discharged electricity to the list.
        df_prosumer['Total battery discharged']= list_total_battery_discharged 

        #Adding prosumer battery allowance to dataframe.
        list_prosumer_max_battery=[]
        for t in instance.T:
            list_prosumer_max_battery.append(instance.prosumer_max_battery[p].value) #Adds the max battery capacity for prosumer at each time period
        df_prosumer['Prosumer max battery allowance']= list_prosumer_max_battery 

        #Adding retailer battery allowance to dataframe.
        list_retailer_max_battery=[]
        for t in instance.T:
            list_retailer_max_battery.append(instance.retailer_max_battery[p].value) #Adds the max battery capacity for prosumer at each time period
        df_prosumer['Retailer max battery allowance']= list_retailer_max_battery 

        list_of_data_frames.append(df_prosumer) #Adding each prosumer's data frame to our list of data frames.

    # Store each prosumers data to an excel sheet.
    #Define file name.
    if timing== 'daily':
        file_name = '/Batteries_Prosumers_solutions.xlsx'
    else:
        file_name = '/' + str(day) + '_Batteries_Prosumers_solutions.xlsx'
   
    writer = pd.ExcelWriter('solution_files/' + year + '/' + timing + '_solutions/' + month + file_name, engine='xlsxwriter')
    for i in instance.I:
        list_of_data_frames[i-1].to_excel(writer, sheet_name='Prosumer'+str(i), index=False)
    writer.save() # Close the Pandas Excel writer and output the Excel file.

### Check charge/discharge of prosumers and retailer

In [11]:
def is_complementary(a,b):
    """
    Return True if one of the two arguments is 0
    """
    if a*b == 0:
        return(True)
    else:
        return(False)

def charge_or_discharge(var1, var2):
    """
    Check if 'var1' and 'var2' are complementary component-wise
    """
    for index in var1:
        if not is_complementary(var1[index].value, var2[index].value):
            return(False, "Index:"+str(index))
    return(True)

In [12]:
print("Prosumers only charge or discharge:", charge_or_discharge(instance.prosumer_charged, instance.prosumer_discharged))
print("Retailer only charge or discharge:", charge_or_discharge(instance.retailer_charged, instance.retailer_discharged))

NameError: name 'instance' is not defined